# Sample accessioning

This example shows the Ovation Service Lab (OSL) APIs for sample accessioning by creating a simple `Requisition` with one sample.

## Setup

In [2]:
import uuid
from datetime import date

from ovation.session import connect

## Session

In [3]:
s = connect(input("Email: "), api='https://services-staging.ovation.io', token='/api/v1/sessions')

Email: barry@ovation.io


## Parameters

Many OSL APIs require the Organization `id`.

In [4]:
organization_id = input('Organization id: ')

Organization id: 156


## Requisition

Create a `container` for the sample (in this case, a Tube). The tube `identifier` and `barcode` will be generated by Ovation. You can supply them using the `identifier` and `barcode` attributes of the `container` if needed:

In [ ]:
tube = s.post(s.entity_path('container'), 
              data={'container': {'type': 'Tube'}},
              params={'organization_id': organization_id})

We need to know which project this Requisition belongs to. You can find projects by ID or `name`:

In [16]:
project = s.get(s.entity_path('project'), 
                params={'q': 'Workflow Testing', # Find project by name
                        'organization_id': organization_id}).projects[0]

Create a `Requisition` and the `Sample`:

In [ ]:
requisition_data = {"identifier": str(uuid.uuid4()), # Any unique (within organization) identifier
                    "template": "Basic",
                    "project_id": project.id,
                    "organization_id": organization_id,
                    "custom_attributes": {'my-attribute': 1.0}, # Optional; Requisition custom attributes
                    "samples": [
                        {"identifier": str(uuid.uuid4()), # Any unique (within organization) identifier
                         "date_received": date.today().isoformat(),
                         "custom_attributes": {'my-sample-attribute': 1.0}, # Optional; Sample custom attributes
                         "sample_states": [
                                {"container_id": tube.id,
                                "position": "A01"}
                            ]
                        }
                    ]
                  }

req = s.post(s.entity_path('requisition'), 
             data={'requisition': requisition_data},
             params={'organization_id': organization_id})